<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/all_in_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
!pip install exchange_calendars

In [110]:
!pip install finance-datareader

In [111]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import nltk
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [112]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [ ]:
class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99], pre_period=0):
    self.period = period
    self.increased = increased
    self.traded = traded
    self.pre_period = pre_period # 시작기점

    self.price_dic = {}

    self.start, self.today = self.get_date()
    self.df = self.get_stock_df()
    self.filtered_df = self.get_filtered_df()
    self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    if self.pre_period != 0:
      today = (datetime.date.today() - datetime.timedelta(days=self.pre_period)).strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_position(self, df):
    return df['Close'][-1]/df['Close'].max()

  def get_future_mdd(self, s):
    peak = s.iloc[0]
    max_drawdown = 0
    for price in s:
        if price > peak:
            peak = price
        drawdown = (peak - price) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown

  def get_hashtags(self, code):
    page = 5
    li = []
    for i in range(1, page):
      url = 'https://finance.naver.com/item/news_news.naver?code=' + code + '&page=' + str(i) + '&sm=title_entity_id.basic&clusterId='
      result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
      bs_obj = BeautifulSoup(result.content, "html.parser")
      tds = bs_obj.find_all('td', {'class': 'title'})
      texts = [td.text.strip() for td in tds]
      li.extend([sentence.replace('[', ' ').replace(']', ' ').replace('…', ' ').replace('·', ' ').replace('"', ' ').replace('“', ' ').replace(',', ' ').replace('...', ' ').replace("'", ' ').replace('‘', ' ').replace('’', ' ') for sentence in texts])

    words = [word for sentence in li for word in sentence.split()]
    stop_words = ['%', '에', '전망', '수혜', '추가', '주가', '주식', '특징주', \
                  '코스닥', '코스피', '시총', '전년비', '1분기', '2분기', '3분기', \
                  '4분기', '전년比', '↑', '지난해', '영업익', '대비', '증가', '영업이익',\
                  '규모', '속', '주당', '매출', '/', '소', '전년', '1Q', '2Q', '3Q', \
                  '4Q', '관련', '데이터로', '보는', '증시', '영향', '등']
    words = [word for word in words if word not in stop_words]
    word_frequencies = nltk.FreqDist(words)
    keywords = word_frequencies.most_common(11)
    return ' '.join(['#'+x[0] for x in keywords[1:]])

  # 위에 고장 시 사용
  def get_hashtags2(self, code):
    name = self.df[self.df['종목코드'] == code]['종목명'].values[0]
    url = 'https://news.google.com/rss/search?q="' + name + '"&hl=ko&gl=KR&ceid=KR%3Ako'
    res = requests.get(url)
    res.encoding = 'utf-8'
    content = res.text
    root = ET.fromstring(content)
    title_tags = root.findall(".//title")
    titles = [title_tag.text for title_tag in title_tags]
    titles = [sentence.replace('[', ' ').replace(']', ' ').replace('…', ' ').replace('·', ' ').replace('"', ' ').replace('“', ' ').replace(',', ' ').replace('...', ' ').replace("'", ' ').replace('‘', ' ').replace('’', ' ') for sentence in titles]
    titles = [title for title in titles if name in title]
    titles = [item.split('-')[0].strip() for item in titles]
    words = [word for sentence in titles for word in sentence.split()]
    stop_words = ['%', '에', '전망', '수혜', '추가', '주가', '주식', '특징주', \
                  '코스닥', '코스피', '시총', '전년비', '1분기', '2분기', '3분기', \
                  '4분기', '전년比', '↑', '지난해', '영업익', '대비', '증가', '영업이익',\
                  '규모', '속', '주당', '매출', '/', '소', '전년', '1Q', '2Q', '3Q', \
                  '4Q', '관련', '데이터로', '보는', '증시', '영향', '등', '이번주', '주목', \
                  '하나증권', '목표주가', '리포트', '프리핑', '목표가', '000원', '장중']
    words = [word for word in words if word not in stop_words]
    word_frequencies = nltk.FreqDist(words)
    keywords = word_frequencies.most_common(11)
    return ' '.join(['#'+x[0] for x in keywords[1:]])

  def get_price_df(self, codes):
    d = {}
    for i in codes:
      d[i] = fdr.DataReader(str(i), self.start, self.today)
    return d

  def get_result_df(self, hashtags=False):
    self.price_dic = self.get_price_df(self.filtered_df['종목코드'])

    if (self.pre_period != 0):
      future_dic = {}
      for i in self.filtered_df['종목코드']:
        future = (datetime.date.today() + datetime.timedelta(days=self.pre_period)).strftime('%Y%m%d')
        future_dic[i] = fdr.DataReader(str(i), self.today, future)
      self.filtered_df['mdd'] = [round(self.get_future_mdd(future_dic[x]['Close']), 4) for x in future_dic]
      self.filtered_df['Performance'] = [round((future_dic[x]['Close'][-1]/future_dic[x]['Close'][0]), 4) for x in future_dic]
      self.filtered_df['preperiod'] = [self.pre_period for x in future_dic]
    else:
      self.filtered_df['mdd'] = [0 for x in self.price_dic]
      self.filtered_df['Performance'] = [0 for x in self.price_dic]
      self.filtered_df['preperiod'] = [self.pre_period for x in self.price_dic]

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Position'] = [self.get_position(self.price_dic[x]) for x in self.price_dic]

    if hashtags == True:
      self.filtered_df['Hashtags'] = [self.get_hashtags(x) for x in self.price_dic.keys()]
    else:
      self.filtered_df['Hashtags'] = [0 for x in self.price_dic.keys()]

    t = self.filtered_df.sort_values('Sharpe', ascending=False)

    scaler = MinMaxScaler(feature_range=(5, 10))

    t['Position'] = scaler.fit_transform(t[['Position']]).round(1)
    t['Sortino'] = scaler.fit_transform(t[['Sortino']]).round(1)
    t['Sharpe'] = scaler.fit_transform(t[['Sharpe']]).round(1)
    t['거래대금'] = scaler.fit_transform(t[['거래대금']]).round(1)
    t['등락률'] = scaler.fit_transform(t[['등락률']]).round(1)
    t['합산'] = t['Position'] + t['Sortino'] + t['Sharpe'] + t['거래대금'] + t['등락률']
    t['합산'] = scaler.fit_transform(t[['합산']]).round(1)
    t['preperiod'] = t[['preperiod']]
    t['Performance'] = t[['Performance']]
    t['mdd'] = t[['mdd']]

    t = t[['종목명', '종료일 종가', '등락률', '거래대금',	'Sharpe',	'Sortino',	'Position', '합산', 'Performance', 'Hashtags', 'preperiod', 'mdd']].reset_index(drop=True).rename({'종료일 종가':'현재가', '등락률':'파워', '거래대금':'관심도', 'Sharpe':'Risk1', 'Sortino':'Risk2', 'Position':'모멘텀'}, axis=1)
    t.index+=1
    t.index.name='순위'

    return t

In [ ]:
# k
stocklist = StockList(365, [.98,1], [.99,1]) # 원래 250인데 365로 바꿈
t = stocklist.filtered_df
t = t.sort_values(['거래대금'], ascending=[False]).head(3)

In [ ]:
# k trend

d = {}
d = stocklist.get_price_df(t['종목코드'].values)

merged_df = pd.DataFrame()
num_intervals = 15

num_intervals = 15

for key, df in d.items():
    increased_ratio = df['Close'] / df['Close'].iloc[0]
    interval_size = int((len(increased_ratio) - 1) / (num_intervals - 1))
    row_indices = list(range(0, len(increased_ratio), interval_size))
    row_indices[-1] = len(increased_ratio) - 1
    selected_ratio = increased_ratio.iloc[row_indices]
    merged_df[key] = selected_ratio

merged_df

,086520,247540,003670
Date,,,
2022-08-04,1.000000,1.000000,1.000000
2022-08-30,1.059569,0.875781,1.239852
2022-09-26,1.230465,0.721094,1.225092
2022-10-21,1.435545,0.831250,1.380074
2022-11-15,1.297857,0.910937,1.741697
2022-12-08,1.182619,0.808594,1.505535
2023-01-03,1.146143,0.728125,1.383764
2023-01-30,1.272822,0.807813,1.608856
2023-02-22,2.387799,1.253906,1.616236


In [ ]:
from google.colab import files

merged_df.to_csv('king_trend_df.csv', encoding="utf-8-sig")
files.download('king_trend_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
t = t[['종목명','등락률','거래대금']].reset_index(drop=True)
t.index+=1
t.index.name='순위'
total_traded = t['거래대금'].sum()
t['거래대금'] = (t['거래대금'] / total_traded) * 100
t

,종목명,등락률,거래대금
순위,,,
1,에코프로,1122.10,41.963304
2,에코프로비엠,198.43,32.137681
3,포스코퓨처엠,252.01,25.899015


In [ ]:
from google.colab import files

t.to_csv('king_df.csv', encoding="utf-8-sig")
files.download('king_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Ai
stocklist = StockList()
t = stocklist.get_result_df(hashtags=True).head(15)
t = t.drop(['preperiod', 'Performance', 'mdd'], axis=1)
t

,종목명,현재가,파워,관심도,Risk1,Risk2,모멘텀,합산,Hashtags
순위,,,,,,,,,
1,이오테크닉스,162300,5.8,5.3,10.0,10.0,9.8,10.0,#신고가 #강세 #반도체 #HBM #52주 #외국인 #순매수 #실적 #장비 #종목
2,HPSP,31650,5.0,7.3,8.6,7.1,9.0,8.3,#반도체 #순매수 #오늘의 #강세 #장비 #브리핑 #오르며 #체결강도 #한미반도체 ...
3,ISC,96200,7.9,5.9,8.3,6.4,9.5,8.8,#테스트 #반도체 #SKC #인수 #브리핑 #비메모리 #외국인 #순매수 #급증 #기업
4,가온칩스,42950,6.0,6.4,8.3,6.8,9.0,8.1,#vs #ARM #삼성전자 #삼성 #파운드리 #여의도 #반도체 #강세 #주식왕 #공...
5,큐렉소,20550,7.9,6.0,8.2,7.4,10.0,9.4,#의료로봇 #수술로봇 #판매 #공급 #인공관절 #올해 #< #최대 #경신 #의료기기
6,실리콘투,8340,8.0,6.3,7.3,6.7,9.6,8.7,#K #글로벌 #신고가 #뷰티누리 #vs #브리핑 #급증 #확대 #세계 #분석
7,삼아알미늄,102200,8.0,6.1,7.2,6.5,8.4,8.0,#강세 #조일알미늄 #양극박 #2차전지 #LG엔솔 #배터리 #오르며 #체결강도 #알...
8,DB,1806,5.0,5.3,7.0,6.5,7.5,5.9,#DB손보 #DB하이텍 #DB손해보험 #실적 #DB금융투자 #구축 #DB메탈 #상반...
9,레이크머티리얼즈,16080,9.1,10.0,7.0,5.8,7.5,9.4,#vs #외국인 #급증 #매도잔량 #순매수 #여의도 #승부사 #여의도를 #달굴 #종목은?


In [ ]:
t.to_csv('ai_df.csv', encoding="utf-8-sig")
files.download('ai_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# leading
# 2주/4주 간격 업데이트
exclude_list = t.head(3)['종목명'].values
leading_df = t[~t['종목명'].isin(exclude_list)]
data = {
    '날짜': ['2022', '2022', '2022'],
    '종목명': leading_df.sort_values('합산', ascending=False).head(3)['종목명'].values,
    '매수가': [0, 0, 0],
    '목표가': [0, 0, 0],
    '손절가': [0, 0, 0],
    '목표수익률': [0, 0, 0],
}
leading_df = pd.DataFrame(data)
leading_df

,날짜,종목명,매수가,목표가,손절가,목표수익률
0,,한미반도체,0,0,0,0
1,,레이크머티리얼즈,0,0,0,0
2,,삼아알미늄,0,0,0,0


In [ ]:
leading_df.to_csv('leading_df.csv', encoding="utf-8-sig")
files.download('leading_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# performance
all_df = []
for p in [20, 60, 120]:
  stocklist = StockList(period=250, increased=[.9, .99], traded=[.9, .99], pre_period=p)
  t = stocklist.get_result_df(hashtags=False).head(15)
  t = t[['종목명', '현재가', 'mdd', 'Performance', 'preperiod']].reset_index(drop=True)
  all_df.append(t)
t = pd.concat(all_df)
t = t.reset_index(drop=True)
t.index.name='순위'
t.index+=1
t

In [ ]:
t.to_csv('performance_df.csv', encoding="utf-8-sig")
files.download('performance_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# sentiments

!pip install newsapi-python
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
from newsapi import NewsApiClient
import datetime
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from wordcloud import WordCloud
import re

api_key = '6147ad35d10843b2949edc41cd955155'
api = NewsApiClient(api_key=api_key)

In [ ]:
# business general

def get_headline_sentiment(category='business'):
    df = api.get_top_headlines(country='us', category=category, page_size=100)
    df = pd.DataFrame(df['articles'])
    df['date'] = pd.to_datetime(df['publishedAt'])
    df['date'] = df['date'].dt.floor('4H')

    analyzer = SentimentIntensityAnalyzer()

    # Vader Polarity
    df = pd.concat([df, pd.DataFrame([analyzer.polarity_scores(text) for text in df['title']])], axis=1)

    # TextBlob Subjectivity
    df['subjectivity'] = [TextBlob(text).sentiment.subjectivity for text in df['title']]
    df = df.fillna(0)
    return df['neg'].mean() * 100 + 50, df['pos'].mean() * 100 + 50, df['compound'].mean() * 100 + 50

def get_keyword_sentiment(keyword='nasdaq'):
  end_date = datetime.datetime.now().strftime('%Y-%m-%d')
  start_date = (datetime.datetime.now() - datetime.timedelta(days=3)).strftime('%Y-%m-%d')

  df = api.get_everything(q=keyword, language='en', page_size=100, from_param=start_date, to=end_date)
  df = pd.DataFrame(df['articles'])
  df['date'] = pd.to_datetime(df['publishedAt'])
  df['date'] = df['date'].dt.floor('4H')

  analyzer = SentimentIntensityAnalyzer()

  df = pd.concat([df, pd.DataFrame([analyzer.polarity_scores(text) for text in df['title']])], axis=1)

  df['subjectivity'] = [TextBlob(text).sentiment.subjectivity for text in df['title']]
  df = df.fillna(0)
  return df['neg'].mean() * 100 + 50, df['pos'].mean() * 100 + 50, df['compound'].mean() * 100 + 50


In [ ]:
business_sentiment = get_headline_sentiment()
general_sentiment = get_headline_sentiment('general')
nasdaq_sentiment = get_keyword_sentiment('Nasdaq')
snp500_sentiment = get_keyword_sentiment('S&P 500')
dowjones_sentiment = get_keyword_sentiment('Dow Jones')

neg = (business_sentiment[0] + general_sentiment[0] + nasdaq_sentiment[0] + snp500_sentiment[0] + dowjones_sentiment[0]) / 5
pos = (business_sentiment[1] + general_sentiment[1] + nasdaq_sentiment[1] + snp500_sentiment[1] + dowjones_sentiment[1]) / 5
compound = (business_sentiment[2] + general_sentiment[2] + nasdaq_sentiment[2] + snp500_sentiment[2] + dowjones_sentiment[2]) / 5

# Optional: If you want to convert neg, pos, compound to integers, you can use round() or int()
neg = int(round(neg))
pos = int(round(pos))
compound = int(round(compound))

In [ ]:
data = {'positive': [34, 42, 34, 33, 11, 77, pos],
        'negative': [12, 35, 12, 55, 11, 13, neg],
        'compound': [22, 10, 6, 10, 23, 84, compound]}

df = pd.DataFrame(data, index=['2022/11/11', '2022/11/12', '2022/11/13', '2022/11/14', '2022/11/15', '2022/11/16', '2022/11/17'])
df.index.name = 'date'
df

,positive,negative,compound
date,,,
2022/11/11,34,12,22
2022/11/12,42,35,10
2022/11/13,34,12,6
2022/11/14,33,55,10
2022/11/15,11,11,23
2022/11/16,77,13,84
2022/11/17,59,56,55


In [ ]:
df.to_csv('sentiment_df.csv', encoding="utf-8-sig")
files.download('sentiment_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [114]:
# word cloud
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.2 MB/s eta 0:00:00


In [115]:
from multiprocessing import Pool
from konlpy.tag import Hannanum
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html

class News:
  def __init__(self):
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_ranking(self, page=1, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    pages = list(range(1,page+1))
    li = []
    for page in pages:
      url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date=' + date + '&page=' + str(page)
      result = requests.get(url, headers = self.headers)
      bs_obj = BeautifulSoup(result.content, "html.parser")

      div = bs_obj.find_all('div', {'class':'hotNewsList'})
      li+=[html.unescape(x['href']) for x in div[0].find_all('a')]
    return li

  def get_article(self, url):
    url = 'https://finance.naver.com/' + url
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    title = ' '.join(bs_obj.find_all('div', {'class': 'article_info'})[0].find_all('h3')[0].get_text().split())
    context = bs_obj.find_all('div', {'class': 'articleCont'})[0]
    try:
      subheading = context.find_all('strong')[0].get_text()
    except:
      subheading = ''
    t = ' '.join(context.get_text().split())
    text = t[:t.find('@')]
    text = text.replace(subheading, '')
    return title, subheading, text

news = News()
news_list = news.get_ranking(5)
with Pool() as p:
    li = p.map(news.get_article, news_list)
li = np.array(li)

dic = {'Title':li[:,0], 'Subheading':li[:,1], 'Text':li[:,2]}
df = pd.DataFrame.from_dict(dic)
df

,Title,Subheading,Text
0,"[단독] '코스닥 시총 5위' 포스코DX, 코스피로 이사간다","이사회 의결 소식에 주가 9% 급등추가 유동성 확보, 변동성 감소 노려",[서울경제] 포스코그룹의 정보기술(IT) 서비스업체 포스코DX(022100)가 유동...
1,"[특징주]파이버프로, 양자컴 상온작동 가능성 후보물질 발견..암자암호통신 핵심부품 ...",,파이버프로 CI [파이낸셜뉴스] 국내 연구진이 상온에서 대규모 '양자 얽힘' 현상을...
2,[특징주] 오염수 방류 소식에 널뛰는 소금株...신성홀딩스·샘표 약세,,소금 걷는 염부 (화성=연합뉴스) 신현우 기자 = 21일 오후 경기 화성시 공생염전...
3,[특징주] 새내기株 스마트레이더시스템 상한가…시총 2000억 돌파,,[서울경제] 지난 22일 코스닥 시장에 상장한 새내기주인 스마트레이더시스템(4249...
4,"[특징주]우리넷, 양자컴 상온 작동 가능성 韓연구진 발견..양자내성암호 개발 상용화...",,이것이 바로 양자컴퓨터 모형 (서울=연합뉴스) 진연수 기자 = 26일 오후 서울 중...
...,...,...,...
120,"자비스, 중기부 46파이 원통형 배터리 검사장비 국책과제 선정",,[이데일리 이정현 기자] 자비스(254120)는 중소벤처기업부 중소기업기술정보진흥원...
121,"aT, 비축기지 2023년 을지연습 실시 및 지속가능한 농업·농촌 모색",,공사 장성비축기지 2023 을지연습 현장. 한국농수산식품유통공사 김춘진 사장(둘째줄...
122,"""재계는 세대교체""…정의선 등 '젊은 회장님'도 수두룩","""해외서 태어난 재계 4~5세 땐 국적 논란 우려""",국내 재계의 세대교체 흐름에 따라 1970년 이후 태어난 오너가(家) 임원 중 회장...
123,‘더 시에나 라이프’ 현대백화점 판교점 팝업스토어…첫 날부터 큰 호응,,명품 브랜드도 아닌데 국내 신규 브랜드의 팝업 스토어에서 제품을 구매하기 위한 웨이...


In [153]:
# 위에 고장 시 사용
import re

# url = 'https://news.google.com/rss/topics/CAAqIggKIhxDQkFTRHdvSkwyMHZNR2RtY0hNekVnSnJieWdBUAE?hl=ko&gl=KR&ceid=KR%3Ako' 경제

url = 'https://news.google.com/rss/topics/CAAqIQgKIhtDQkFTRGdvSUwyMHZNREpmTjNRU0FtdHZLQUFQAQ?hl=ko&gl=KR&ceid=KR%3Ako' # 금융
res = requests.get(url)
res.encoding = 'utf-8'
content = res.text
root = ET.fromstring(content)
title_tags = root.findall(".//title")
titles = [title_tag.text for title_tag in title_tags]
titles = [sentence.replace('↓', ' ').replace('↓',' ').replace('(', ' ').replace(')', ' ').replace('[', ' ').replace(']', ' ').replace('…', ' ').replace('·', ' ').replace('"', ' ').replace('“', ' ').replace(',', ' ').replace('...', ' ').replace("'", ' ').replace('‘', ' ').replace('’', ' ').replace('”', ' ').replace('＂', ' ').replace('?', ' ').replace('‥', ' ').replace('!', ' ') for sentence in titles]
titles = [item.split('-')[0].strip() for item in titles]

In [154]:
cleaned = [re.split(r'\s*[-/|ㅣ]\s*', title)[0].strip() for title in titles]
cleaned = [' '.join([word for word in item.split() if not any(keyword in word for keyword in ['경제', '뉴스', '신문'])]) for item in cleaned[1:]]
# df = pd.DataFrame({'Title': cleaned})
# df

,Title
0,토큰포스트 브리핑 친리플 변호사 SEC 소송 XRP에 악영향 소송 무기로 사용 外
1,엇갈리는 BTC 전망 2만달러 밑 vs 상승 앞둬
2,단독 코스닥 시총 5위 포스코DX 코스피로 이사간다
3,에코프로 포스코 하락 예상한다면 2차전지 인버스 ETF 나온다
4,결국 사적 연금으로 각자도생 재정 안정 방점 찍은 국민연금
...,...
65,亞증시
66,국민연금 운용역 성과급 주자 반대의견 나오는 이유는
67,가계대출 고삐 죄나 금감원 은행권 대출 종합점검할 것
68,한화 장남 김동관이 스톡옵션 대신 받은 RS 요즘 대세라는데


In [155]:
import random
import re

hannanum = Hannanum()

def clean_title(title):
    cleaned_title = title.replace(',', ' ') \
                         .replace('(', ' ') \
                         .replace(')', ' ') \
                         .replace('…', ' ') \
                         .replace('?', ' ') \
                         .replace('.', ' ') \
                         .replace("'", ' ') \
                         .replace('"', ' ') \
                         .replace('”', ' ') \
                         .replace('“', ' ') \
                         .replace('‘', ' ') \
                         .replace('’', ' ') \
                         .replace('·', ' ') \
                         .replace('[', ' ') \
                         .replace(']', ' ') \
                         .replace('↓', ' ') \
                         .replace('→', ' ') \
                         .replace('↑', ' ')
    return cleaned_title

def rank_words(words_list):
  word_ranking = {}
  for word in words_list:
      if word in word_ranking:
          word_ranking[word] += 1
      else:
          word_ranking[word] = 1
  sorted_word_ranking = dict(sorted(word_ranking.items(), key=lambda item: item[1], reverse=True))
  return sorted_word_ranking

def standardize_values(dictionary):
    max_value = max(dictionary.values())
    min_value = min(dictionary.values())

    for key in dictionary:
        value = dictionary[key]
        standardized_value = 10 + (value - min_value) * (90 / (max_value - min_value))
        dictionary[key] = round(standardized_value)

    return dictionary

def custom_standardize_values(dictionary, new_min, new_max, noise_min, noise_max):
    max_value = max(dictionary.values())
    min_value = min(dictionary.values())

    standardized_dict = {}
    for key, value in dictionary.items():
        # Linear transformation to scale the values
        standardized_value = new_min + (value - min_value) * (new_max - new_min) / (max_value - min_value)

        # Add random noise
        noise = random.uniform(noise_min, noise_max)
        standardized_value_with_noise = standardized_value + noise

        standardized_dict[key] = round(standardized_value_with_noise, 0)

    return standardized_dict

def extract_nouns(text, custom_nouns_list):
    words = re.findall(r'\b\w+\b', text)

    filtered_nouns = {}
    for word in words:
        if word in custom_nouns_list:
            filtered_nouns[word] = filtered_nouns.get(word, 0) + 1

    return filtered_nouns

In [156]:
df['Cleaned_Title'] = df['Title'].apply(clean_title)
df['Filtered_Pos'] = [hannanum.nouns(x) for x in df['Cleaned_Title']]
words = [word for sublist in df['Filtered_Pos'] for word in sublist]
word_frequency_dict = rank_words(words)
result = standardize_values(word_frequency_dict)

In [157]:
stop_words = ['의', '등', '때', '오늘', '이곳', '저', '월', '년', '일', '로', '어디', '데', '특징주', '장중', '전', '주', '위', '한', '수',' 속', '전', '추', '속', '내', '비트코']
data_dict = {key: value for key, value in result.items() if key not in stop_words}
data_dict = {key: value for key, value in data_dict.items() if not str(key).isdigit()}

In [158]:
df

,Title,Cleaned_Title,Filtered_Pos
0,토큰포스트 브리핑 친리플 변호사 SEC 소송 XRP에 악영향 소송 무기로 사용 外,토큰포스트 브리핑 친리플 변호사 SEC 소송 XRP에 악영향 소송 무기로 사용 外,"[토큰포스트, 브리핑, 친리플, 변호사, 소송, 악영향, 소송, 무기, 사용, 外]"
1,엇갈리는 BTC 전망 2만달러 밑 vs 상승 앞둬,엇갈리는 BTC 전망 2만달러 밑 vs 상승 앞둬,"[전망, 2만달러, 밑, 상승]"
2,단독 코스닥 시총 5위 포스코DX 코스피로 이사간다,단독 코스닥 시총 5위 포스코DX 코스피로 이사간다,"[단독, 코스닥, 시총, 5위, 포스코DX, 코스피]"
3,에코프로 포스코 하락 예상한다면 2차전지 인버스 ETF 나온다,에코프로 포스코 하락 예상한다면 2차전지 인버스 ETF 나온다,"[에코프, 포스코, 하락, 예상, 2차전지, 인버스]"
4,결국 사적 연금으로 각자도생 재정 안정 방점 찍은 국민연금,결국 사적 연금으로 각자도생 재정 안정 방점 찍은 국민연금,"[사적, 연금, 각자도생, 재정, 안정, 방점, 국민연금]"
...,...,...,...
65,亞증시,亞증시,[亞증시]
66,국민연금 운용역 성과급 주자 반대의견 나오는 이유는,국민연금 운용역 성과급 주자 반대의견 나오는 이유는,"[국민연금, 운용역, 성과급, 반대의견, 이유]"
67,가계대출 고삐 죄나 금감원 은행권 대출 종합점검할 것,가계대출 고삐 죄나 금감원 은행권 대출 종합점검할 것,"[가계대출, 고삐, 죄, 금감원, 은행권, 대출, 종합점검, 것]"
68,한화 장남 김동관이 스톡옵션 대신 받은 RS 요즘 대세라는데,한화 장남 김동관이 스톡옵션 대신 받은 RS 요즘 대세라는데,"[한화, 장남, 김동관, 스톡옵션, 대, 대, 데]"


In [159]:
#설정 종목명

title_as_sentence = ' '.join(df['Cleaned_Title'].values)
custom_nouns = stocklist.df['종목명'].values.tolist() + ['비트코인', '이더리움']
filtered_nouns = extract_nouns(title_as_sentence, custom_nouns)
ticker_dict = custom_standardize_values(filtered_nouns, 40, 80, 5, 15)

In [160]:
data_dict.update(ticker_dict)

In [161]:
t = pd.DataFrame(data_dict, index=['value']).T.sort_values(by='value', ascending=False).head(50)

In [162]:
t

,value
비트코인,90.0
국민연금,82.0
포스코DX,54.0
유한양행,53.0
디알텍,52.0
HMM,50.0
한화,50.0
이더리움,50.0
에코프로,49.0
SK렌터카,48.0


In [ ]:
from google.colab import files

t.to_csv('words_df.csv', header=False, encoding="utf-8-sig")
files.download('words_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>